Chiến lược trend following thực hiện theo mô hình PPF

Step 1 get data from yfinance

In [11]:
import yfinance as yf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
import numpy as np
import talib as ta
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.models import load_model


data = yf.download('USDJPY=X', start='2023-01-01', interval='1h')

data.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-01-02 00:00:00+00:00,130.886002,131.041000,130.850006,130.903000,130.903000,0
2023-01-02 01:00:00+00:00,130.906006,130.957001,130.826996,130.917999,130.917999,0
2023-01-02 02:00:00+00:00,130.910004,131.007004,130.897003,131.001999,131.001999,0
2023-01-02 03:00:00+00:00,130.992996,131.003006,130.873993,130.897003,130.897003,0
2023-01-02 04:00:00+00:00,130.889999,130.932007,130.865997,130.895004,130.895004,0


Step 2. Thực hiện tính các Indicator và clean data:

In [12]:
# Tính RSI
data['RSI'] = ta.RSI(data['Close'], timeperiod=14)

# Tính EMA
data['EMA'] = ta.EMA(data['Close'], timeperiod=21)

# Tính EMA của RSI
data['EMA(RSI)'] = ta.EMA(data['RSI'], timeperiod=14)

# Tính AU và AD
def calculate_au_ad(data, window=14):
    delta = data['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    return avg_gain, avg_loss

data['AU'], data['AD'] = calculate_au_ad(data)

# In ra dữ liệu đã được tính toán chỉ số kỹ thuật

data = data.dropna()
print(data.head())

                                 Open        High         Low       Close  \
Datetime                                                                    
2023-01-03 03:00:00+00:00  129.919998  129.983994  129.649994  129.778000   
2023-01-03 04:00:00+00:00  129.778000  129.798996  129.487000  129.591995   
2023-01-03 05:00:00+00:00  129.591995  129.951996  129.520004  129.813995   
2023-01-03 06:00:00+00:00  129.813004  130.059998  129.589996  129.835999   
2023-01-03 07:00:00+00:00  129.841995  130.251999  129.750000  130.110992   

                            Adj Close  Volume        RSI         EMA  \
Datetime                                                               
2023-01-03 03:00:00+00:00  129.778000       0  16.753430  130.575963   
2023-01-03 04:00:00+00:00  129.591995       0  14.636242  130.486511   
2023-01-03 05:00:00+00:00  129.813995       0  26.564471  130.425373   
2023-01-03 06:00:00+00:00  129.835999       0  27.643656  130.371794   
2023-01-03 07:00:00+00:00  1

Step 3. Xây dựng mô hình PPF

In [13]:
# Xác định trạng thái thị trường dựa trên sự thay đổi giá so với EMA
data['Price_vs_EMA'] = data['Close'] - data['EMA']

def determine_trend(price_vs_ema):
    if price_vs_ema > 0:
        return 1
    elif price_vs_ema < 0:
        return -1
    else:
        return 0
    
data['Trend'] = data['Price_vs_EMA'].apply(determine_trend)

# Xóa cột Price_vs_EMA nếu bạn không cần nữa
data.drop(columns=['Price_vs_EMA'], inplace=True)

print(data.head())

                                 Open        High         Low       Close  \
Datetime                                                                    
2023-01-03 03:00:00+00:00  129.919998  129.983994  129.649994  129.778000   
2023-01-03 04:00:00+00:00  129.778000  129.798996  129.487000  129.591995   
2023-01-03 05:00:00+00:00  129.591995  129.951996  129.520004  129.813995   
2023-01-03 06:00:00+00:00  129.813004  130.059998  129.589996  129.835999   
2023-01-03 07:00:00+00:00  129.841995  130.251999  129.750000  130.110992   

                            Adj Close  Volume        RSI         EMA  \
Datetime                                                               
2023-01-03 03:00:00+00:00  129.778000       0  16.753430  130.575963   
2023-01-03 04:00:00+00:00  129.591995       0  14.636242  130.486511   
2023-01-03 05:00:00+00:00  129.813995       0  26.564471  130.425373   
2023-01-03 06:00:00+00:00  129.835999       0  27.643656  130.371794   
2023-01-03 07:00:00+00:00  1

/var/folders/tj/qj_bq7f53g34hd4pn6vrnggw0000gp/T/ipykernel_2015/2035968506.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Price_vs_EMA'] = data['Close'] - data['EMA']
/var/folders/tj/qj_bq7f53g34hd4pn6vrnggw0000gp/T/ipykernel_2015/2035968506.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Trend'] = data['Price_vs_EMA'].apply(determine_trend)
/var/folders/tj/qj_bq7f53g34hd4pn6vrnggw0000gp/T/ipykernel_2015/2035968506.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy o

Thực hiện huấn luyện mô hình

In [14]:
min = 1
for i in range(15):
    # Chọn đặc trưng và biến mục tiêu
    features = ['EMA', 'AU', 'AD']
    target = 'Market_State'

    # Loại bỏ dữ liệu bị thiếu
    data.dropna(inplace=True)

    # Chọn các đặc trưng
    features = [ 'EMA', 'AU', 'AD']

    # Chuẩn hóa dữ liệu
    scaler = StandardScaler()
    X = scaler.fit_transform(data[features])

    # Chuyển đổi nhãn thành số
    encoder = LabelEncoder()
    y = encoder.fit_transform(data['Trend'])

    # Chia dữ liệu thành tập huấn luyện và tập kiểm tra
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Xây dựng mô hình mạng nơ-ron
    model = Sequential()
    model.add(Dense(64, input_dim=len(features), activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(11, activation='relu'))
    model.add(Dense(3, activation='softmax'))

    # Compile mô hình
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Huấn luyện mô hình
    history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

    # Đánh giá hiệu suất
    test_loss, test_accuracy = model.evaluate(X_test, y_test)

    # In ra kết quả đánh giá
    print(f'Test Loss: {test_loss}')
    print(f'Test Accuracy: {test_accuracy}')

    if(test_loss < min): 
        min = test_loss
        model.save('dff_forex.h5')
        # In ra kết quả đánh giá
        print(f'Test Loss: {test_loss}')
        print(f'Test Accuracy: {test_accuracy}')

/var/folders/tj/qj_bq7f53g34hd4pn6vrnggw0000gp/T/ipykernel_2015/2536781759.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7471 - loss: 0.5720 - val_accuracy: 0.8486 - val_loss: 0.3373
Epoch 2/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8593 - loss: 0.3106 - val_accuracy: 0.8492 - val_loss: 0.3357
Epoch 3/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8529 - loss: 0.3131 - val_accuracy: 0.8385 - val_loss: 0.3612
Epoch 4/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8575 - loss: 0.3165 - val_accuracy: 0.8492 - val_loss: 0.3361
Epoch 5/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8679 - loss: 0.2929 - val_accuracy: 0.8510 - val_loss: 0.3339
Epoch 6/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8635 - loss: 0.3027 - val_accuracy: 0.8450 - val_loss: 0.3393
Epoch 7/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8679 - loss: 0.2947 - val_accuracy: 0.8361 - val_loss: 0.3605
Epoch 8/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8645 - loss: 0.3053 - val_accuracy: 0.

Test Loss: 0.34371936321258545
Test Accuracy: 0.8450118899345398
Test Loss: 0.34371936321258545
Test Accuracy: 0.8450118899345398
Epoch 1/10


/var/folders/tj/qj_bq7f53g34hd4pn6vrnggw0000gp/T/ipykernel_2015/2536781759.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


211/211 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8192 - loss: 0.5552 - val_accuracy: 0.8337 - val_loss: 0.3672
Epoch 2/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8531 - loss: 0.3208 - val_accuracy: 0.8420 - val_loss: 0.3428
Epoch 3/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8709 - loss: 0.2949 - val_accuracy: 0.8403 - val_loss: 0.3488
Epoch 4/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8653 - loss: 0.2979 - val_accuracy: 0.8367 - val_loss: 0.3518
Epoch 5/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8651 - loss: 0.2990 - val_accuracy: 0.8492 - val_loss: 0.3445
Epoch 6/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8674 - loss: 0.3002 - val_accuracy: 0.8510 - val_loss: 0.3347
Epoch 7/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8789 - loss: 0.2919 - val_accuracy: 0.8438 - val_loss: 0.3389
Epoch 8/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8687 - loss: 0.2981 - val_accuracy: 0.8480 - val_

...

211/211 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6900 - loss: 0.7159 - val_accuracy: 0.8492 - val_loss: 0.3410
Epoch 2/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8671 - loss: 0.3067 - val_accuracy: 0.8367 - val_loss: 0.3519
Epoch 3/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8535 - loss: 0.3181 - val_accuracy: 0.8432 - val_loss: 0.3429
Epoch 4/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8586 - loss: 0.3077 - val_accuracy: 0.8444 - val_loss: 0.3387
Epoch 5/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8620 - loss: 0.3005 - val_accuracy: 0.8474 - val_loss: 0.3415
Epoch 6/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8628 - loss: 0.2978 - val_accuracy: 0.8355 - val_loss: 0.3612
Epoch 7/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8665 - loss: 0.2980 - val_accuracy: 0.8486 - val_loss: 0.3427
Epoch 8/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8641 - loss: 0.3008 - val_accuracy: 0.8486 - val_

/var/folders/tj/qj_bq7f53g34hd4pn6vrnggw0000gp/T/ipykernel_2015/2536781759.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


211/211 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7880 - loss: 0.5776 - val_accuracy: 0.8379 - val_loss: 0.3608
Epoch 2/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8531 - loss: 0.3136 - val_accuracy: 0.8450 - val_loss: 0.3394
Epoch 3/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8561 - loss: 0.3149 - val_accuracy: 0.8438 - val_loss: 0.3411
Epoch 4/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8570 - loss: 0.3140 - val_accuracy: 0.8409 - val_loss: 0.3475
Epoch 5/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8691 - loss: 0.2985 - val_accuracy: 0.8450 - val_loss: 0.3388
Epoch 6/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8660 - loss: 0.2973 - val_accuracy: 0.8432 - val_loss: 0.3363
Epoch 7/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8669 - loss: 0.2915 - val_accuracy: 0.8480 - val_loss: 0.3372
Epoch 8/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8644 - loss: 0.3077 - val_accuracy: 0.8480 - val_

/var/folders/tj/qj_bq7f53g34hd4pn6vrnggw0000gp/T/ipykernel_2015/2536781759.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


211/211 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7907 - loss: 0.6182 - val_accuracy: 0.8337 - val_loss: 0.3695
Epoch 2/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8517 - loss: 0.3178 - val_accuracy: 0.8462 - val_loss: 0.3489
Epoch 3/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8617 - loss: 0.3028 - val_accuracy: 0.8504 - val_loss: 0.3360
Epoch 4/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8697 - loss: 0.2921 - val_accuracy: 0.8468 - val_loss: 0.3425
Epoch 5/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8549 - loss: 0.3201 - val_accuracy: 0.8296 - val_loss: 0.3742
Epoch 6/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8549 - loss: 0.3119 - val_accuracy: 0.8373 - val_loss: 0.3612
Epoch 7/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8703 - loss: 0.2929 - val_accuracy: 0.8438 - val_loss: 0.3360
Epoch 8/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8643 - loss: 0.3039 - val_accuracy: 0.8492 - val_